### Set path to original pyNeuroChem. Please change to your own path

#### Import required libraries

In [1]:
import sys
#sys.path.append('/home/olexandr/notebooks/ASE_ANI/lib')
from ase_interface import ANIENS # ANI -> ASE Interface class
from ase_interface import aniensloader #  ANI ensemble loader function

van Der Waals correction will be unavailable. Please install ased3


In [2]:
import numpy as np
import  ase
#from ase.build import molecule
#from ase.neb import NEB
from ase.calculators.mopac import MOPAC

#from ase.neb import NEBtools
from ase.io import read, write

import matplotlib
import matplotlib as mpl

import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline

#### Load the 8x network ANI-1ccx ensemble
Preloading the ensemble allows a single class to be used in a for loop or multiple classes without being reloaded

In [3]:
# Current ANI model options are:
# '../ani_models/ani-1ccx_8x.info' Coupled cluster transfer learned model
# '../ani_models/ani-1x_8x.info'   Full ANI-1x wb97x/6-31g* dataset model
aniensemble = aniensloader('../ani_models/ani-1ccx_8x.info', 0, multigpu=False)

#### Reading IRC datra for one reaction (Cope rearrangement of citral). Look also in data folder for more examples

In [4]:
traj = read('data/citralOCopeALL.xyz', index=':')

In [5]:
len(traj)

200

In [6]:
energies = []
energies_mop = []
atomic_energies = []
D1 =[]
D2 = []
for geometry in traj:
    geometry.set_calculator(ANIENS(aniensemble))
    energies.append(geometry.get_potential_energy())
    
    #calculating atomic enegry decomposition 
    atomic_energies.append(geometry.calc.get_atomicenergies())
    
    # monitoring certain distances relevant to the reaction
    D1.append(geometry.get_distance(0, 1))
    D2.append(geometry.get_distance(6, 9))    
    geometry.set_calculator(MOPAC(method='PM7'))
    energies_mop.append(geometry.get_potential_energy())

RuntimeError: mopac in . returned an error: 127

In [ ]:
energies = np.array(energies)
energies_mop = np.array(energies_mop)
dE_mop =  energies_mop - energies_mop.min()
dE =  energies - energies.min()

In [ ]:
mpl.rcParams['figure.figsize'] = (10.0, 7.0)
fig = plt.figure()
fig.suptitle('Reaction bond Distances', fontsize=14, fontweight='bold')
ax = fig.add_subplot(111)
ax.plot(dE*23.06, label='ANI')
ax.plot(dE_mop*23.06, label='PM7')
plt.legend(fontsize=24)
sns.set(font_scale=1.0)
plt.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Frame', fontsize=28)
ax.set_ylabel('E, kcal/mol', fontsize=28)

plt.show()

In [ ]:
mpl.rcParams['figure.figsize'] = (10.0, 7.0)
fig = plt.figure()
fig.suptitle('Reaction bond distance profile', fontsize=14, fontweight='bold')
ax = fig.add_subplot(111)
ax.plot(D1, label='R(A,B)')
ax.plot(D2, label='R(C,D)')
plt.legend(fontsize=24)
sns.set(font_scale=1.0)
plt.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Frame', fontsize=28)
ax.set_ylabel('R, Angstrom', fontsize=28)

plt.show()

# Atomic energy decomposition

In [ ]:
atomic_e =  np.array(atomic_energies)
atomic_e.shape

In [ ]:
mpl.rcParams['figure.figsize'] = (10.0, 7.0)
fig = plt.figure()
fig.suptitle('Reaction local AEV energies', fontsize=14, fontweight='bold')
ax = fig.add_subplot(111)
ax.plot(atomic_e[:,0], label='E(A)')
ax.plot(atomic_e[:,1], label='E(B)')
plt.legend(fontsize=24)
sns.set(font_scale=1.0)
plt.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Frame', fontsize=28)
ax.set_ylabel('E, a.u', fontsize=28)

plt.show()

In [ ]:
mpl.rcParams['figure.figsize'] = (10.0, 7.0)
fig = plt.figure()
fig.suptitle('Reaction local AEV energies', fontsize=14, fontweight='bold')
ax = fig.add_subplot(111)
ax.plot(atomic_e[:,6], label='E(C)')
ax.plot(atomic_e[:,9], label='E(D)')
plt.legend(fontsize=24)
sns.set(font_scale=1.0)
plt.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Frame', fontsize=28)
ax.set_ylabel('E, a.u', fontsize=28)

plt.show()